In [4]:
from importlib import reload ,import_module
import module.utilize as utilize
import module.multiVariant as multiVariant
import module.singleVariant as singleVariant
import module.multiHistogramBase as multiHistogramBase
import numpy as np
from numba import njit,jit, float32
import module.singleVariantCopulaBase as CopulaBase
from tqdm import tqdm
import time
from multiprocessing import Pool
from sklearn.metrics import root_mean_squared_error
import cupy as cp
import module.multiHistogramSparse as multiHistogramSparse
reload(utilize)
reload(multiVariant)
reload(singleVariant)
reload(multiHistogramBase)
reload(CopulaBase)
reload(multiHistogramSparse)
import module.ananlysisFuncion as analysisFunction


startTime=time.time()

#attribute_names=np.array(["phi_grav","particle_mass_density","zmom","ymom"])
attribute_names=np.array(["phi_grav","particle_mass_density"])
incremental_number=300
all_ensamble_data=np.empty([0,incremental_number,64,64,64])

for name in attribute_names:
    data=utilize.readFiles(name,incremental_number)
    data=data.reshape(1,incremental_number,64,64,64)
    all_ensamble_data=np.append(all_ensamble_data,data,axis=0)

#print(all_ensamble_data.shape)
#print(all_ensamble_data[0].shape)
covBlockSize=5
dataBlockSize=5
binsNumber=128
sizeZ=60
sizeY=60
sizeX=60
minMaxBlockSize=2
isMinMax=False

conditions=np.array([[0,1e5],[3e10,4e10]])

print("start fit model")
with tqdm(total=4, desc="Model fitting") as pbar:

    oursModel=multiVariant.multiDistCopula3D.load(f"Nyx_{attribute_names.shape[0]}varaibles_{incremental_number}members_128Bins_dBlock5_cBlock5_new")
    print("ours complete fit")
    pbar.update(1)
    copulaBlockSize=2
    copulaBaseModel=CopulaBase.multiVariantCopulaBase(all_ensamble_data,copulaBlockSize)
    copulaBaseModel.fit()
    print("copula complete fit")
    pbar.update(1)
    multiHistBlockSize=2
    multiHistModel=multiHistogramSparse.multiHistogramSpaseModel(all_ensamble_data,blockSize=multiHistBlockSize,binsNumber=binsNumber)
    multiHistModel.fit()
    print("multi-hist complete fit")
    pbar.update(1)
    gtModel=multiHistogramSparse.multiHistogramSpaseModel(all_ensamble_data,blockSize=1,binsNumber=binsNumber)
    gtModel.fit()

    multiBinEdges=gtModel.vBinEdges

    print("complete fit")
    pbar.update(1)


result=np.zeros([sizeZ,sizeY,sizeX],dtype=np.float32)
oursResult=np.zeros([sizeZ,sizeY,sizeX],dtype=np.float32)
copulaBaseResult=np.zeros([sizeZ,sizeY,sizeX],dtype=np.float32)
mtResult=np.zeros([sizeZ,sizeY,sizeX],dtype=np.float32)

with tqdm(total=sizeZ*sizeY*sizeX, desc="總進度") as pbar:
    for idx in range(sizeZ * sizeY * sizeX):
        
        z = idx // (sizeY * sizeX)
        y = (idx // sizeX) % sizeY
        x = idx % sizeX      

        #Ground Truth
        samples=gtModel.sampleByPos(z,y,x) 
        prob=analysisFunction.probability_in_range_numba(samples,conditions)
        result[z,y,x]=prob

        #Ours
        samples=oursModel.sampleByPos(z,y,x)
        prob=analysisFunction.probability_in_range_numba(samples,conditions)
        oursResult[z,y,x]=prob
        
        
        #Copula base
        samples=copulaBaseModel.sampleByPos(z,y,x)
        prob=analysisFunction.probability_in_range_numba(samples,conditions)
        copulaBaseResult[z,y,x]=prob

        samples=multiHistModel.sampleByPos(z,y,x)
        prob=analysisFunction.probability_in_range_numba(samples,conditions)
        mtResult[z,y,x]=prob

        pbar.update(1)

result.tofile(f"NyxGT_{attribute_names.shape[0]}varaibles.bin")
oursResult.tofile(f"NyxOus_{attribute_names.shape[0]}varaibles.bin")
copulaBaseResult.tofile(f"NyxCopulaBase_{attribute_names.shape[0]}varaibles.bin")
mtResult.tofile(f"NyxMtResult_{attribute_names.shape[0]}varaibles.bin")

start fit model


Model fitting:   0%|          | 0/4 [00:00<?, ?it/s]

ours complete fit


Model fitting:  50%|█████     | 2/4 [00:55<00:55, 27.98s/it]

copula complete fit


Model fitting:  75%|███████▌  | 3/4 [01:43<00:36, 36.19s/it]

multi-hist complete fit


Model fitting: 100%|██████████| 4/4 [02:37<00:00, 39.48s/it]


complete fit


總進度: 100%|██████████| 216000/216000 [19:08<00:00, 188.04it/s]
